#### CellStrat Prompt Engineering for Generative AI - Module 2 : Applications Of Prompt Engineering


In [5]:
import google.generativeai as genai
import textwrap

import google.generativeai as genai
import yaml
from yaml import SafeLoader

import os
from dotenv import load_dotenv

from IPython.display import display
from IPython.display import Markdown
from google.api_core import retry, exceptions
from google.api_core import retry


In [2]:
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
model = genai.GenerativeModel('models/gemini-1.5-flash')


In [7]:
# Automatic retry
model = genai.GenerativeModel("gemini-1.5-flash-latest")
prompt = "Write a story about a magic backpack."

model.generate_content(
    prompt, request_options={"retry": retry.Retry(predicate=retry.if_transient_error)}
)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The backpack was old, worn leather that creaked like a rusty hinge. Found tucked away in the dusty attic of her grandmother\u2019s house, Elara had always been drawn to it. It seemed to hum with a low, quiet energy, a whisper of something magical. \n\nHer grandmother, a woman with eyes like distant stars, had only smiled enigmatically when Elara asked about it. \"It's a keeper, child,\" she\u2019d said, her voice a soft rustle of leaves. \"A friend for adventures.\"\n\nElara, ever the curious one, was determined to discover its secret. She tried everything - poking, prodding, even whispering spells she'd found in a dusty book. Nothing. But the backpack always felt warm to the touch, a comforting weight on her shoulder.\n\nOne day, while walking through the b

Manually increase timeout when responses take time
If you encounter ReadTimeout or DeadlineExceeded errors, meaning an API call exceeds the default timeout (600 seconds), you can manually adjust it by defining timeout in the request_options argument.

In [8]:
model = genai.GenerativeModel("gemini-1.5-flash-latest")
prompt = "Write a story about a magic backpack."

model.generate_content(
    prompt, request_options={"timeout": 90}
)  # Increase timeout to  90 sec

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Elara, a girl with eyes the color of storm clouds and a spirit as wild as the wind, lived in a village nestled in the foothills of a mountain range that whispered ancient secrets. Elara dreamt of exploring the world, of seeing beyond the familiar faces and the worn paths of her home. But her family was poor, and travel was a luxury they couldn't afford.\n\nOne day, rummaging through her grandmother's attic, Elara stumbled upon a dusty, leather backpack tucked away in a forgotten corner. It was intricately woven with silver thread, depicting fantastical creatures that seemed to shimmer with life. As Elara reached for it, a sudden gust of wind swept through the attic, causing a lamp to topple over and illuminate the backpack in a peculiar glow.\n\nIntrigued, E

# Manual Implementation

In [9]:

model = genai.GenerativeModel("gemini-1.5-flash-latest")


@retry.Retry(
    predicate=retry.if_transient_error,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def generate_with_retry(model, prompt):
    response = model.generate_content(prompt)
    return response


prompt = "Write a one-liner advertisement for magic backpack."

generate_with_retry(model=model, prompt=prompt)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "## Carry more than you ever thought possible. Introducing the Magic Backpack! \ud83c\udf92\u2728 \n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
        

# Test the error handling with retry mechanism
- To validate that your error handling and retry mechanism work as intended, define a generate_content function that deliberately raises a ServiceUnavailable error on the first call. This setup will help you ensure that the retry decorator successfully handles the transient error and retries the operation.

In [10]:
from google.api_core import retry, exceptions


@retry.Retry(
    predicate=retry.if_transient_error,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def generate_content_first_fail(model, prompt):
    if not hasattr(generate_content_first_fail, "call_counter"):
        generate_content_first_fail.call_counter = 0

    generate_content_first_fail.call_counter += 1

    try:
        if generate_content_first_fail.call_counter == 1:
            raise exceptions.ServiceUnavailable("Service Unavailable")

        response = model.generate_content(prompt)
        return response.text
    except exceptions.ServiceUnavailable as e:
        print(f"Error: {e}")
        raise


model = genai.GenerativeModel("gemini-1.5-flash-latest")
prompt = "Write a one-liner advertisement for magic backpack."

generate_content_first_fail(model=model, prompt=prompt)
     

Error: 503 Service Unavailable


'**Carry your world, not the weight of it.** (with an image of the magic backpack) \n'